# To train and test Yolov5, the first clone the repository. All dependencies must be installed an imported as well

In [ ]:
!pip install roboflow
#export.py requires this version of tensorflow. In the future, other versions may work, however this one works as of 11/11
!pip install tensorflow==2.6.2

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt
import pathlib
import os
import cv2
import zipfile
from os import path
import shutil
import csv
import random
import numpy as np
import configparser

# The next cells contain necessary steps to train the model on the Kitti dataset

In [3]:
#creates a yaml file to train YOLOv5 on the Kitti dataset
data = open("data.yaml", "w")
data.write("train: ./kittiyolo/train/images\nval: ./kittiyolo/valid/images\nnc: 8\nnames: [\'Car\', \'Pedestrian\', \'Cyclist\', \'Van\', \'Truck\', \'Person_sitting\', \'Tram\', \'DontCare\']")
data.close()

NOTE: As noted in the README, in order to download the Kitti dataset and labels correctly, you must modify the config.ini file with the appropriate links. These links can be accessed by registering for an account at http://www.cvlibs.net/datasets/kitti/user_login.php

In [ ]:
parser = configparser.ConfigParser()
if os.path.isfile('config.ini'):
  parser.read('config.ini')
  images_link = parser.get('Kitti', 'KittiImagesLink')
  labels_link = parser.get('Kitti', 'KittiLabelsLink')
else:
  print('Config file not found')

In [1]:
#downloads the kitti images in a zip file
# NOTE: this will take a lot of space
!wget {images_link} -O kitti.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
#downloads the kitti labels in a zip file
!wget {labels_link} -O kittilabel.zip

In [6]:
#unzips both the images and labels
with zipfile.ZipFile("./kitti.zip", 'r') as zip_ref:
    zip_ref.extractall("kitti")

import zipfile
with zipfile.ZipFile("./kittilabel.zip", 'r') as zip_ref:
    zip_ref.extractall("kittilabel")

The next few boxes convert the original labels of the Kitti dataset to a form that YOLOv5 is able to use. If you wish to understand this transformation, open a Kitti label file before running the next few cells, and then open the corresponding file after running the cells.

In [7]:
data_dir = pathlib.Path('kitti/training/image_2')
label_dir = pathlib.Path('kitti/training/label_2')
yolo_labels = pathlib.Path('yolo')

In [8]:
image_count = len(list(data_dir.glob('*.png')))
label_count = len(list(label_dir.glob('*.txt')))

In [9]:
root_path = "./kitti/training/"
yolo_path = "yolo/"

In [10]:
kitti2yolotype_dict = {
    "Car": "0",
    "Van": "0",
    "Pedestrian": "1",
    "Person_sitting": "1",
    "Cyclist": "2",
    "Truck": "3",
    "Tram": "6",
    "Misc": "6",
    "DontCare": "6",
}
def kitti2yolo(kitti_label, img_height, img_width):

    kitti_label_arr = kitti_label.split(" ")
    x1 = float(kitti_label_arr[4])
    y1 = float(kitti_label_arr[5])
    x2 = float(kitti_label_arr[6])
    y2 = float(kitti_label_arr[7])

    bb_width = x2 - x1
    bb_height = y2 - y1
    yolo_x = (x1 + 0.5 * bb_width) / img_width
    yolo_y = (y1 + 0.5 * bb_height) / img_height
    yolo_bb_width = bb_width / img_width
    yolo_bb_height = bb_height / img_height
    yolo_label = kitti2yolotype_dict[kitti_label_arr[0]]

    return (
        yolo_label
        + " "
        + str(yolo_x)
        + " "
        + str(yolo_y)
        + " "
        + str(yolo_bb_width)
        + " "
        + str(yolo_bb_height)
    )

In [11]:
%cp -r kittilabel/training/label_2 kitti/training/label_2

In [12]:
kitti_images_path = root_path + "image_2" + os.sep
kitti_labels_path = root_path + "label_2" + os.sep

# yolo paths
if yolo_path is None:
    yolo_path = root_path + "yolo_labels" + os.sep

if not os.path.exists(yolo_path):
    os.makedirs(yolo_path)

# load each kitti label, convert to yolo and save
for labelfilename in os.listdir(kitti_labels_path):
  yolo_labels = []
  with open(kitti_labels_path + labelfilename, "r") as kittilabelfile:
    cvimage = cv2.imread(
        kitti_images_path + labelfilename.split(".txt")[0] + ".png"
        )
    height, width, frame_depth = cvimage.shape
    for kitti_label in kittilabelfile:
      yolo_labels.append(
          kitti2yolo(kitti_label, img_height=height, img_width=width)
          )
      with open(yolo_path + labelfilename, "w+") as yololabelfile:
        for label in yolo_labels:
          yololabelfile.write(label + "\n")

The next cells move the correct YOLOv5 labels to a directory so they can be used to train the model

In [ ]:
%mkdir kittiyolo
%cd kittiyolo
%mkdir train
%mkdir valid
%cd valid
%mkdir images
%mkdir labels
%cd ../..

In [14]:
%cp -r yolo kittiyolo/train/labels
%cp -r kitti/training/image_2 kittiyolo/train/images

In [15]:
root_path = "./kittiyolo/train/"
kittiyolo_images_path = root_path + "images" + os.sep
kittiyolo_labels_path = root_path + "labels" + os.sep
yolo_valid_path = "./yolo/valid"

# load each kitti label, convert to yolo and save
count = 0
for labelfilename in os.listdir(kittiyolo_images_path):
  count = count+1
  if count == 3:
    break

In [16]:
def moveFiles(src, dst):
  files = []
  for i in os.listdir(src):
    files.append(i)
  files = sorted(files)
  listt = np.random.RandomState(seed=42).permutation(files)[:1000]
  for f in listt:
    shutil.copy(os.path.join(src, f), dst)
    p = os.path.join(src, f)
    os.remove(p)

In [17]:
moveFiles("kittiyolo/train/images", "kittiyolo/valid/images")
moveFiles("kittiyolo/train/labels", "kittiyolo/valid/labels")

# The next cells add a dataset of about 900 custom images to the Kitti dataset. 
This allows for detection of pedestrians at a closer range, as the original Kitti dataset only contains images of pedestrians from a distance. If you wish to only train on Kitti, skip these cells. This custom dataset was created and is housed in Roboflow. Access to this dataset will need to be obtained if you wish to edit or view the dataset more fully.

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="v40ZMYu9O9KrUmkE9j9O")
project = rf.workspace().project("528.2-project")
dataset = project.version(2).download("yolov5")

In [19]:
# In the Roboflow dataset, only three classes exist, whereas Kitti attempts to label
# 8 classes. The only class number that exists incorrectly in the Roboflow dataset
# is the truck label, which is 4 in the Kitti dataset, but originally 2 in the 
# Roboflow dataset. This method performs a conversion
def convert_trucks():
  for filename in os.listdir(os.getcwd()):
      newlines = []
      flag = False
      with open(os.path.join(os.getcwd(), filename), "r") as f:
          if str(filename[0]) == "p":
              flag = True
              lines = f.readlines()
              for x in lines:
                  if x[0] == "2":
                      newlines.append("4" + x[1:])
                  else:
                      newlines.append(x)
      f.close()
      if flag:
          with open(os.path.join(os.getcwd(), filename), "w") as f:
              for x in newlines:
                  f.write("%s" % x)
          f.close()
      flag = False

In [ ]:
%cd ./528.2-Project-2/train/labels
convert_trucks()

In [ ]:
%cd ../../valid/labels
convert_trucks()

In [ ]:
%cd ../../..

In [23]:
moveFiles('528.2-Project-2/train/labels', 'kittiyolo/train/labels')
moveFiles('528.2-Project-2/train/images', 'kittiyolo/train/images')
moveFiles('528.2-Project-2/valid/labels', 'kittiyolo/valid/labels')
moveFiles('528.2-Project-2/valid/images', 'kittiyolo/valid/images')

# The next cell can be used to train the dataset on the small version of YOLOv5. 
Changing the parameters might lead to improved performance. See the detect.py file for information on parameters

In [ ]:
!python train.py --img 512 --batch 100 --epochs 25 --data ./data.yaml --weights yolov5s.pt

# The cells below can be used to create an 8 bit integer quantized tflite model
If you ran the above code to train the model, it will be saved in ./runs/train/exp#/best-int8.tflite

You may find it helpful to rename this file a more descriptive name. If you do this, the --weights path in the next few cells must be changed to that path

In [ ]:
!python export.py --weights ./kitti_kaleo_512.pt --include tflite --img-size 448 --int8

The next three cells will create an edge TPU optimized version of the tflite model ready for deployment

In [ ]:
%env TFLITE_FILE=./runs/train/exp/weights/best-int8.tflite

In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

In [ ]:
! edgetpu_compiler $TFLITE_FILE

The next cell evaluates the baseline Pytorch model

In [ ]:
!python val.py --weights ./runs/train/exp/weights/best.pt --img 640 --conf 0.25 --data data.yaml

In [ ]:
!python val.py --weights ./runs/train/exp/weights/best-int8_edgetpu.tflite --img 320 --conf 0.25 --data data.yaml

In [ ]:
!python val.py --weights ./runs/train/exp/weights/best-int8.tflite --img 320 --conf 0.25 --data data.yaml

In [ ]:
!python val.py --weights ./kitti_kaleo_512-int8.tflite --img 448 --conf 0.25 --data data.yaml